In [1]:
from nixtlats import TimeGPT

timegpt = TimeGPT(
    # defaults to os.environ.get("TIMEGPT_TOKEN")
    token = 'JxQF7I2vM8Hpbue718L2gGCfG1gflGqWJCkWb6gRjVL2r9p0TPhDmoOIMAAOgGdQG1tFUDFKgkq52cbyAM1H8Li1jTg9KtV6zWKGuejojqMV6i5wzTgppL8KnsvfcfpYidXbBRgqSLlVilfjjvWwTIMFixvHbIu1cEBdP8CUcjeRlsWlg2ZkYlLr3iiG2RojTAG4BKHRnSX2S7uIpaKMjBZcwRlVpf8KoUFXfAaQj2a5UW98o5IMPe196Iz2uqwD'
)


Nixtla's SDK for TimeGPT takes pandas DF as inputs. Here we are using an Electricity Price Forecasting (EPF) dataset.

The input data will require the following columns:

series identifier (column name specified with #id_col argument)
datestamps (column name specified with #time_col argument)
target historical values (column name specified with #target_col argument)

In [2]:
timegpt.validate_token()


INFO:nixtlats.timegpt:Happy Forecasting! :), If you have questions or need support, please email ops@nixtla.io


True

In [13]:
from utils import data_handling, helpers

window_size = 96
pred_length = pred_length = (96, 192, 336, 720)

data_dict = data_handling.load_electricity()
_, _, dataloader_test = data_handling.convert_data(data_dict, window_size, pred_length)


Feature batch shape: torch.Size([32, 96, 348])


In [42]:
input, target = next(iter(dataloader_test))
input[0,:]
#target[0][0,:]

torch.Size([96, 348])

In [89]:
import torch
import pandas as pd

import pandas as pd
import numpy as np
import torch

# Simulate a PyTorch tensor with random data
tensor = torch.rand(96, 348)

# Convert the tensor to a numpy array and then flatten it
targets = input[0,:].numpy().flatten()

# Create a list of IDs repeated for each time step
ids = np.tile(np.arange(1, 349), 96)  # IDs from 1 to 348

# Create a list of time steps for each ID
time_steps = np.repeat(np.arange(96), 348)

# Create a DataFrame
df = pd.DataFrame({
    'Time': time_steps,
    'ID': ids,
    'Target': targets
})

# The DataFrame has one 'Time' column, one 'ID' column, and one 'Target' column
small_df = df[df["ID"] < 5]
small_df

small_df["Time"] = small_df["Time"] +1
small_df


small_df["Time"] = small_df["Time"].astype(str).copy()

small_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 384 entries, 0 to 33063
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    384 non-null    object 
 1   ID      384 non-null    int64  
 2   Target  384 non-null    float32
dtypes: float32(1), int64(1), object(1)
memory usage: 10.5+ KB


/tmp/ipykernel_59906/221893856.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_df["Time"] = small_df["Time"] +1
/tmp/ipykernel_59906/221893856.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_df["Time"] = small_df["Time"].astype(str).copy()


In [90]:
timegpt_fcst_df = timegpt.forecast(df=small_df, h=96, freq="H", finetune_steps=10, id_col='ID', time_col='Time', target_col='Target')


#fcst_df = timegpt.forecast(df=df, h=36, model='timegpt-1-long-horizon', time_col='timestamp', target_col='value')


INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
/vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/python3.10/site-packages/nixtlats/timegpt.py:242: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["ds"] = pd.to_datetime(df["ds"])


ValueError: Given date string "1" not likely a datetime, at position 0

In [ ]:
helpers.mean_squared_error(output, target)